# The Sudoku Problem Formulation for the PulP Modeller

### Authors: Antony Phillips, Stuart Mitchell

Import `Pulp` modeler functions

In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, lpSum, value

We create a list of strings from "1" to "9".  The `Vals`, `Rows` and `Cols` sequences all follow the same form.

In [2]:
Sequence = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

Vals = Sequence
Rows = Sequence
Cols = Sequence

We create a list (`Boxes`) with the row and column index of each square in each box.

In [3]:
Boxes =[]
for i in range(3):
    for j in range(3):
        Boxes += [[(Rows[3*i+k],Cols[3*j+l]) for k in range(3) for l in range(3)]]

We store the data for this problem in `prob`, a `Pulp.LpProblem` structure. We craft its variables from `Vals`, `Rows` and `Cols`, and we add an arbitrary objective function, say $f(\boldsymbol{x})=0$.   

In [11]:
prob = LpProblem("Sudoku Problem", LpMinimize)

choices = LpVariable.dicts("Choice", (Vals,Rows,Cols), 0, 1, LpInteger)

prob += 0,"Arbitrary Objective Function"

We proceed to include all the constraints:
$$ \sum_{k=1}^9 \sum_{\ell=1}^9 x_{j,k,l} = 1 \text{ for all }1\leq j \leq 9 \text{ (the pilars add up to }1)$$
$$ \sum_{j=1}^9 \sum_{\ell=1}^9 x_{j,k,l} = 1 \text{ for all }1\leq k \leq 9 \text{ (the rows add up to }1)$$
$$ \sum_{j=1}^9 \sum_{k=1}^9 x_{j,k,l} = 1 \text{ for all }1\leq \ell \leq 9 \text{ (the columns add up to }1)$$
$$ \sum_{k=3\lambda}^{3(\lambda+1)} \sum_{\ell=3\mu}^{3(\mu+1)} x_{j,k,l} = 1 \text{ for all } 1\leq j \leq 9 \text{ and all }0 \leq \lambda,\mu \leq 2 \text{ (those }3\times 3 \text{ boxes add up to }1)$$

In [12]:
for r in Rows:
    for c in Cols:
        prob += lpSum([choices[v][r][c] for v in Vals]) == 1,""
        
for v in Vals:
    for r in Rows:
        prob += lpSum([choices[v][r][c] for c in Cols]) == 1,""        
    for c in Cols:
        prob += lpSum([choices[v][r][c] for r in Rows]) == 1,""
    for b in Boxes:
        prob += lpSum([choices[v][r][c] for (r,c) in b]) == 1,""

And finally, each starting number on the Sudoku puzzle gives us an equality constraint:

![](/Users/francisco/Dropbox/Documents/Teaching/FA17/MA524/Extras/wikisudokuproblem.jpg "blah")

In [13]:
prob += choices["7"]["1"]["1"] == 1,""
prob += choices["5"]["2"]["2"] == 1,""
prob += choices["9"]["3"]["6"] == 1,""
prob += choices["4"]["1"]["5"] == 1,""
prob += choices["8"]["1"]["6"] == 1,""
prob += choices["2"]["2"]["8"] == 1,""
prob += choices["4"]["2"]["9"] == 1,""
prob += choices["1"]["3"]["9"] == 1,""
prob += choices["2"]["4"]["2"] == 1,""
prob += choices["5"]["4"]["8"] == 1,""
prob += choices["3"]["5"]["1"] == 1,""
prob += choices["9"]["5"]["3"] == 1,""
prob += choices["5"]["5"]["5"] == 1,""
prob += choices["6"]["5"]["9"] == 1,""
prob += choices["4"]["6"]["4"] == 1,""
prob += choices["7"]["6"]["5"] == 1,""
prob += choices["3"]["6"]["8"] == 1,""
prob += choices["1"]["7"]["5"] == 1,""
prob += choices["4"]["7"]["8"] == 1,""
prob += choices["1"]["8"]["1"] == 1,""
prob += choices["8"]["8"]["2"] == 1,""
prob += choices["6"]["8"]["7"] == 1,""
prob += choices["9"]["8"]["8"] == 1,""
prob += choices["2"]["9"]["1"] == 1,""
prob += choices["7"]["9"]["4"] == 1,""

And that's all.  We can now solve with `prob.solve()` and proceed to print the results.

In [14]:
prob.solve()

1

In [15]:
stringout = ""
for r in Rows:
    if r == "1" or r == "4" or r == "7": 
        print("+-------+-------+-------+")
    for c in Cols:
        for v in Vals:
            if value(choices[v][r][c])==1:
                if c == "1" or c == "4" or c =="7": 
                    stringout += "| "
                stringout += v + " "
                if c == "9": 
                    print(stringout + "|")
                    stringout = ""
print("+-------+-------+-------+") 

+-------+-------+-------+
| 7 3 1 | 2 4 8 | 9 6 5 |
| 9 5 8 | 1 6 7 | 3 2 4 |
| 6 4 2 | 5 3 9 | 7 8 1 |
+-------+-------+-------+
| 4 2 7 | 6 9 3 | 1 5 8 |
| 3 1 9 | 8 5 2 | 4 7 6 |
| 8 6 5 | 4 7 1 | 2 3 9 |
+-------+-------+-------+
| 5 7 3 | 9 1 6 | 8 4 2 |
| 1 8 4 | 3 2 5 | 6 9 7 |
| 2 9 6 | 7 8 4 | 5 1 3 |
+-------+-------+-------+
